# Setup

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from jupyter_bbox_widget import BBoxWidget
from plotly.express import imshow as imshow_interactive

In [ ]:
import sys
import pathlib

REPO_DIR = pathlib.Path(__name__).resolve().parent.parent
SRC_DIR = str(REPO_DIR / "src")
sys.path.append(SRC_DIR)

In [ ]:
from scanplot.io import load_image, dump_coords_csv
from scanplot.plotting import draw_image, draw_ROI
from scanplot.view import (
    CoordinatesMapper, DetectorWidget, MarkerSelectorBBoxWidget, ROISelectorBBoxWidget,
    DetectorWidgetCombined
)

from scanplot.core.process_template import extract_markers_from_image

from scanplot.core import (
    template_match,
    replace_black_pixels,
    normalize_map,
    image_tresholding,
    get_template_mask,
    center_object_on_template_image,
    reconstruct_template_mask,
    generalized_hough_transform,
    CoordinatesConverter,
    bboxes_to_roi,
    _apply_roi,
)

In [ ]:
from scanplot.core import Plot

# Algorithm Pipeline

## 1. Upload plot image

Input data examples can be found in `<REPOSITORY>/datasets/`.

In [ ]:
data_path = pathlib.Path("../datasets/")
plot_number = 22
plot_image_filepath = data_path / "plot_images" / f"plot{plot_number}.png"

In [ ]:
image = load_image(plot_image_filepath)

plot = Plot(image)
plot.draw()

## 2. Select markers on image


Specify how much marker types on image

In [ ]:
plot.set_markers_number(3)

In [ ]:
marker_selector_widget = MarkerSelectorBBoxWidget(
    image_data=plot.data,
    markers_number=plot.markers_number,
)

In [ ]:
display(marker_selector_widget)

Validate chosen bounding boxes and extract marker images

In [ ]:
marker_selector_widget.bboxes = [{'x': 329, 'y': 806, 'width': 64, 'height': 64, 'label': 'marker1'},
 {'x': 262, 'y': 754, 'width': 62, 'height': 52, 'label': 'marker2'}]

In [ ]:
marker_selector_widget.validate_bboxes()

plot.extract_markers(marker_bboxes=marker_selector_widget.bboxes)

In [ ]:
# ## draw extracted markers

# for marker_label, marker_image in plot.markers.items():
#     draw_image(marker_image)
#     plt.title(marker_label)
#     plt.show()

## 3. (optional step) Select region of interest

For each marker type you can specify its own region of interest.
Default ROI is a whole image.

In [ ]:
roi_widget = ROISelectorBBoxWidget(
    image_data=plot.data, 
    markers_number=plot.markers_number,
)

In [ ]:
display(roi_widget)

In [ ]:
# roi_widget.bboxes = [{'x': 457, 'y': 641, 'width': 220, 'height': 139, 'label': 'ROI for marker1'},
#  {'x': 637, 'y': 575, 'width': 147, 'height': 100, 'label': 'ROI for marker1'},
#  {'x': 742, 'y': 478, 'width': 231, 'height': 116, 'label': 'ROI for marker1'},
#  {'x': 746, 'y': 555, 'width': 257, 'height': 242, 'label': 'ROI for marker2'}]

roi_widget.bboxes = [{'x': 238,
  'y': 263,
  'width': 1199,
  'height': 620,
  'label': 'ROI for marker1'},
 {'x': 253,
  'y': 221,
  'width': 1202,
  'height': 666,
  'label': 'ROI for marker2'}]

In [ ]:
plot._images_algorithm_input

In [ ]:
plot.apply_roi(roi_bboxes=roi_widget.bboxes)

In [ ]:
plot.draw_region_of_interest(marker="marker3")

## 4. Run matching algorithms

In [ ]:
correlation_maps = plot.run_matching()

In [ ]:
# imshow_interactive(correlation_maps["marker2"])

In [ ]:
# res = plot._match_single_marker(
#     plot.data,
#     plot.markers["marker1"],
#     plot._marker_masks["marker1"],
# )

## 5. Select algorithm parameters

In [ ]:
settings = {
    "fig_size": 9,
    "marker_size": 70,
    "marker_color": "yellow",
    "marker_type": "*",
}

In [ ]:
# detector_widget_marker1 = DetectorWidget(plot, marker="marker1")
# # detector_widget_marker1.apply_widget_settings(**settings)  # change settings if you want
# detector_widget_marker1.widget()

In [ ]:
detector_widget = DetectorWidgetCombined(plot)
detector_widget.apply_widget_settings(**settings)   # change settings if you want
detector_widget.widget()

In [ ]:
# points coordinates in pixels (!)

all_detections = detector_widget.get_detections()

marker1_detections = all_detections["marker1"]
x = marker1_detections[:, 0]
y = marker1_detections[:, 1]
plt.scatter(x, y, alpha=0.5)

## 5. Convert obtained coordinates from pixel to real values

In [ ]:
mapper = CoordinatesMapper(image)

fig_size = 10
mapper_widget = mapper.interactive_widget(fig_size=fig_size)

In [ ]:
display(mapper_widget)

Convert pixel coordinates to factual coordinates

In [ ]:
converter = CoordinatesConverter()
converter.import_parameters_from_mapper(mapper)

In [ ]:
x_px = marker1_detections.x
y_px = marker2_detections.y

x_factual, y_factual = converter.from_pixel(x_pixel=x_px, y_pixel=y_px)

In [ ]:
plt.scatter(x_factual, y_factual)

Save obtained coordinates in csv

In [ ]:
dump_coords_csv(
    x=x_factual,
    y=y_factual,
    savepath=f"detections_plot{PLOT_NUMBER}_marker{MARKER_NUMBER}.csv"
)